In [ ]:
!pip install transformers einops accelerate bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64

In [ ]:
checkpoint = "MBZUAI/LaMini-T5-738M"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.float32)

In [ ]:
!pip install langchain langchain_community

In [ ]:
from langchain.llms import HuggingFacePipeline

In [ ]:
def llm_pipeline():
    pipe = pipeline(
            'text2text-generation',
            model = base_model,
            tokenizer = tokenizer,
            max_length = 256,
            do_sample = True,
            temperature = 0.3,
            top_p = 0.95
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

In [ ]:
input_prompt = "Write an article on generative AI"

In [ ]:
model = llm_pipeline()
generated_text = model(input_prompt)
generated_text

In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'MBZUAI/LaMini-T5-738M',
    'HF_TASK':'text2text-generation',
    'device_map':'auto',
    'torch_dtype':'torch.float32'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="2.2.0"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g4dn.xlarge",
	container_startup_health_check_timeout=300,
  )

# send request
predictor.predict({
	"inputs": "Write desription on quantum computers",
})

In [ ]:
prompt = "Write a short article on Quantum Computers"

# hyper-parameter payload
payload = {
        "inputs":prompt,
        "parameters":{
            "do_sample":True,
            "top_p":0.7,
            "temperature":0.3,
            "top_k":50,
            "max_new_tokens":500,
            "repetition_penalty":1.03
        }
}

response = predictor.predict(payload)
print(response)

In [ ]:
ENDPOINT = "huggingface-pytorch-tgi-inference-2024-08-28-10-44-21-362"

In [ ]:
import boto3

runtime = boto3.client('runtime.sagemaker')

response = runtime.invoke_endpoint(EndpointName=ENDPOINT, ContentType="application/json", Body=json.dumps(payload))

In [ ]:
prediction = json.loads(response['Body'].read().decode('utf-8'))

In [ ]:
prediction

In [ ]:
prediction[0]['generated_text']